In [3]:
#!pip install crewai

  Using cached crewai-0.134.0-py3-none-any.whl.metadata (35 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.10.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-1.0.13-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached instructor-1.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached litellm-1.72.0-py3-none-any.whl.metadata (39 kB)
  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached openai-1.93.0-py3-none-any.whl.metadata (29 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http

In [13]:
import os
from crewai import Agent, Task, Crew, LLM
#from langchain_ollama import ChatOllama
from langchain_community.chat_models import ChatOllama
from crewai.cli.constants import ENV_VARS

# Override the key name dynamically
# for entry in ENV_VARS.get("ollama", []):
#     if "API_BASE" in entry:
#         entry["BASE_URL"] = entry.pop("API_BASE")  # Rename the key


In [2]:
!pip install --upgrade groq

In [ ]:
#!pip install langchain-ollama


In [14]:
!ollama list


NAME                        ID              SIZE      MODIFIED    
gemma3:1b                   8648f39daa8f    815 MB    4 days ago     
mxbai-embed-large:latest    468836162de7    669 MB    11 days ago    
codellama:latest            8fdf8f752f6e    3.8 GB    2 weeks ago    
phi4-mini:latest            78fad5d182a7    2.5 GB    3 weeks ago    
codellama:7b                8fdf8f752f6e    3.8 GB    6 weeks ago    


In [4]:
#llm = ChatOllama(model="gemma3:1b")

llm = LLM(
    api_key=os.environ["GROQ_API_KEY"],
    model= "groq/llama3-70b-8192"
)

In [5]:
frontendEngineer = Agent(
    role=" Front-end Engineer",
    goal="to code exclusive UI according to customer's description of the UI: {description}",
    backstory="you are a working in a startup that develops extraordinary websites for clients. Your tech stack is HTML, CSS, JS only.",
    allow_delegation=False,
    verbose=True,
    llm = llm
) 

In [6]:
backendEngineer = Agent(
    role="Backend Engineer ",
    goal="to write efficient backend according to the frontend developed by the front-end engineer",
    backstory="you are a backend engineer working at a startup that develops extraordinary websites for clients. You use FastAPI for developing backend.",
    allow_delegation=False,
    verbose=True,
    llm = llm
) 

In [7]:
qaTester = Agent(
     role="Quality Assurance Tester",
    goal="to check the code after the final code reaches you from the backend engineer. You make sure it works and is efficient.",
    backstory="you are a Quality Assurance Tester working at a startup. Your job is to make sure that the final code is reliable and working.",
    allow_delegation=True,
    verbose=True,
    llm = llm
    
)

In [8]:
frontend = Task(
    description=("Write front-end code using html, css, js according to the user description: {description}."),
    expected_output="Clean efficient and modern front-end. You give your final code after rechecking for mistakes and errors. Also write comments too. Make sure to write the code.",
    agent=frontendEngineer
    
)

In [9]:
backend = Task(
    description=("Write backend code in FastAPI using Python and connect it to the front end."),
    expected_output="clean efficient and working backend with comments explaining stuff. Also give the front end provided by frontend engineer.",
    agent=backendEngineer
    
)

In [10]:
QA = Task(
    description=("check the code after both frontend and backend are completed. Make sure it works well. Otherwise, ask the frontend engineer and backend engineer to fix the code."),
    expected_output="Confirmation that the code is complete, then write the final code once.",
    agent=qaTester
    
)

In [11]:
crew = Crew(
    agents = [frontendEngineer, backendEngineer, qaTester],
    tasks = [frontend, backend, QA],
    verbose =1,
    memory=False
)

In [12]:
type(llm)

crewai.llm.LLM

In [82]:

result = crew.kickoff(inputs = {"description": "A cool portfolio website along with a blog page"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: da3ff4a3-bbcf-4d47-9da4-24d183644b0a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:  Front-end Engineer                                                                                     │
│                                                                                                                 │
│  Task: Write front-end code using html, css, js according to the user description: A cool portfolio website     │
│  along with a blog page.                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:  Front-end Engineer                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  I now can give a great answer                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 51bc7a01-8577-44cb-ba6d-0ae366e5d6a7                                                                     │
│  Agent:  Front-end Engineer                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Backend Engineer                                                                                        │
│                                                                                                                 │
│  Task: Write backend code in FastAPI using Python and connect it to the front end.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate 
limit reached for model `llama3-70b-8192` in organization `org_01jxprten0fc2tpwy47r12egaz` service tier `on_demand`
on tokens per minute (TPM): Limit 6000, Used 3880, Requested 2160. Please try again in 391.999999ms. Need more 
tokens? Upgrade to Dev Tier today at 
https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 3ebed652-c1e4-41c3-9469-8c8dc0216093                                                                     │
│  Agent: Quality Assurance Tester                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: da3ff4a3-bbcf-4d47-9da4-24d183644b0a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-70b-8192` in organization `org_01jxprten0fc2tpwy47r12egaz` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 3880, Requested 2160. Please try again in 391.999999ms. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
